# Compile a file that has domain qry rank

In [48]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

In [49]:
results = pd.read_csv("/net/lazer/lab-lazer/shared_projects/google_audit_reproduce/intermedidate_files/merged_summary/day_qry_domain_rank.csv")

In [50]:
results

,qry,domain,serp_rank,counts
0,Abby Finkenauer,abbyfinkenauer.com,1.0,21886
1,Abby Finkenauer,abbyfinkenauer.com,2.0,10425
2,Abby Finkenauer,abbyfinkenauer.com,3.0,7054
3,Abby Finkenauer,abbyfinkenauer.com,4.0,387
4,Abby Finkenauer,abbyfinkenauer.com,5.0,3847
...,...,...,...,...
222825,Zoe Lofgren,zoelofgren.com,19.0,3361
222826,Zoe Lofgren,zoelofgren.com,20.0,27
222827,Zoe Lofgren,zoelofgren.com,21.0,447
222828,Zoe Lofgren,zoelofgren.com,22.0,27


## merge with partisanship

In [51]:
politician_info_dir = "../data/qry_info_house_filter.csv"
politician_info = pd.read_csv(politician_info_dir)

In [52]:
qry_party_df = pd.merge(results, politician_info[['qry', 'party']].drop_duplicates(), how="left", on='qry')

In [53]:
qry_party_df['counts'].sum()

299573378

In [54]:
qry_party_df

,qry,domain,serp_rank,counts,party
0,Abby Finkenauer,abbyfinkenauer.com,1.0,21886,Democrat
1,Abby Finkenauer,abbyfinkenauer.com,2.0,10425,Democrat
2,Abby Finkenauer,abbyfinkenauer.com,3.0,7054,Democrat
3,Abby Finkenauer,abbyfinkenauer.com,4.0,387,Democrat
4,Abby Finkenauer,abbyfinkenauer.com,5.0,3847,Democrat
...,...,...,...,...,...
222825,Zoe Lofgren,zoelofgren.com,19.0,3361,Democrat
222826,Zoe Lofgren,zoelofgren.com,20.0,27,Democrat
222827,Zoe Lofgren,zoelofgren.com,21.0,447,Democrat
222828,Zoe Lofgren,zoelofgren.com,22.0,27,Democrat


In [56]:
party_domain_rank = qry_party_df.groupby(['domain', 'party', 'serp_rank'], dropna=True, as_index=False)['counts'].sum()

In [57]:
party_domain_rank

,domain,party,serp_rank,counts
0,1011now.com,Democrat,6.0,401
1,1011now.com,Democrat,7.0,808
2,1011now.com,Democrat,8.0,1235
3,1011now.com,Democrat,9.0,1747
4,1011now.com,Democrat,10.0,416
...,...,...,...,...
74660,zoelofgren.com,Democrat,19.0,3361
74661,zoelofgren.com,Democrat,20.0,27
74662,zoelofgren.com,Democrat,21.0,447
74663,zoelofgren.com,Democrat,22.0,27


In [58]:
party_domain_rank['counts'].sum()

299573378

In [59]:
party_domain_rank.to_csv('../data/house_analysis/party_domain_rank.csv', index=False)  

# local vs. naitonal

In [60]:
domain_classification = pd.read_csv("../data/domain_classification_local_national.csv")

In [61]:
hand_label = pd.read_csv("../data/domains_handcode_local_national_labels.csv")

In [62]:
hand_label = hand_label[['domain', 'label']][~hand_label['label'].isna()]

In [63]:
party_domain_rank = pd.merge(party_domain_rank, domain_classification, how="left", on=['domain'])

In [64]:
party_domain_rank = pd.merge(party_domain_rank, hand_label.drop_duplicates(), how="left", on=['domain'])

In [65]:
party_domain_rank['classification'] = party_domain_rank['classification'].fillna(party_domain_rank['label'])

In [66]:
party_domain_rank.loc[party_domain_rank["classification"] == "INCONSISTENT", "classification"] = "other news"

In [67]:
party_domain_rank.loc[party_domain_rank["classification"] == "international", "classification"] = "national"

In [68]:
party_domain_rank['counts'].sum()

299573378

## add category

In [69]:
allison_cat = pd.read_csv("../data/domain_category.csv",)

In [70]:
party_domain_rank = pd.merge(party_domain_rank, allison_cat.drop_duplicates(), how="left", on=['domain'])

In [71]:
party_domain_rank['counts'].sum()

299573378

## add is labels

In [72]:
def check_is_news(col):
    if col == "news":
        return True
    else:
        return False

In [73]:
party_domain_rank['is_news'] = party_domain_rank['category'].apply(check_is_news)

In [74]:
def check_is_local(col):
    if pd.isna(col):
        return None
    elif col == "local":
        return True
    else:
        return False

In [75]:
party_domain_rank['is_local'] = party_domain_rank['classification'].apply(check_is_local)

In [76]:
def is_poli_controlled(row):
    if row['category']=='campaign_personal':
        return True
    elif row['category']=='gov':
        if row['domain']=='house.gov':
            return True
        else:
            return False
    elif row['category']=='social':
        if row['domain']=='youtube.com':
            return False
        else:
            return True
    else:
        return False

In [77]:
party_domain_rank['is_poli_control'] = party_domain_rank.apply(is_poli_controlled, axis=1)

In [79]:
party_domain_rank['rank'] = party_domain_rank['serp_rank'] +1

In [80]:
# Drop a single column
party_domain_rank = party_domain_rank.drop('label', axis=1)

In [81]:
party_domain_rank

,domain,party,serp_rank,counts,classification,category,is_news,is_local,is_poli_control,rank
0,1011now.com,Democrat,6.0,401,local,news,True,True,False,7.0
1,1011now.com,Democrat,7.0,808,local,news,True,True,False,8.0
2,1011now.com,Democrat,8.0,1235,local,news,True,True,False,9.0
3,1011now.com,Democrat,9.0,1747,local,news,True,True,False,10.0
4,1011now.com,Democrat,10.0,416,local,news,True,True,False,11.0
...,...,...,...,...,...,...,...,...,...,...
74660,zoelofgren.com,Democrat,19.0,3361,NaN,campaign_personal,False,None,True,20.0
74661,zoelofgren.com,Democrat,20.0,27,NaN,campaign_personal,False,None,True,21.0
74662,zoelofgren.com,Democrat,21.0,447,NaN,campaign_personal,False,None,True,22.0
74663,zoelofgren.com,Democrat,22.0,27,NaN,campaign_personal,False,None,True,23.0


In [82]:
party_domain_rank.to_csv("../data/house_analysis/party_domain_rank_class_cat_isnews_islocal_ispolicontrol.csv", index=False)